In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
import pickle
from mosaicml import *
from mosaicml.constants import MLModelFlavours

In [ ]:
df = pd.read_csv("/data/winequality.csv")
df.head(10)

In [ ]:
# The target variable is 'quality'.
Y = df['quality']
X = df[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar','chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density','pH', 'sulphates', 'alcohol']]


# Split the data into train and test data:
from sklearn.model_selection import train_test_split
import os

#split_percentage = float(os.getenv("split_percentage"))
split_percentage = 0.2
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = split_percentage)
X_train = X_train.to_numpy()
Y_train = Y_train.to_numpy()


# Use ML Algorithm
from sklearn.ensemble import RandomForestRegressor

my_model = RandomForestRegressor(max_depth=6, random_state=0, n_estimators=10)
my_model.fit(X_train,Y_train)

@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data_list = payload
    data_array = np.asarray(data_list)
    try:
        prediction = model.predict(data_array)
    except:
        prediction = model.predict(data_array.reshape(1, -1))
    return prediction.tolist()

In [ ]:
import requests
req = requests.Request()
req.json = {"payload":[ 7.4   ,  0.35  ,  0.33  ,  2.4   ,  0.068 ,  9    , 26    ,
        0.9947,  3.36  ,  0.6   , 11.9   ]}

score(my_model,req)

In [ ]:
register_model(my_model,
                score,
                "wine_quality_regression_notebook",
                "wine_quality_regression_notebook",
                MLModelFlavours.sklearn,
                init_script="mkdir abc \\n cd abc \\n touch abc.txt",
                input_type="json",
                explain_ai=True,
                x_train=X_train, 
                y_train=Y_train, 
                feature_names=['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar','chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density','pH', 'sulphates', 'alcohol'], 
                feature_ids=['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar','chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density','pH', 'sulphates', 'alcohol'],
                target_names=["wine_quality"]
              )